### Initialize
Using the NeuroConv environment

In [1]:
from decode_lab_code.utils import nwb_utils
from decode_lab_code.utils import signal_utils
import matplotlib.pyplot as plt

Cite NWB
Cite CatalystNeuro: NeuroConv toolbox if converting Neuralynx data


### Load in data


In [ ]:
folder_path = '/Users/js0403/local data/2021-03-31_08-59-02 16eB R1 10min rec after sec drive cells on 2 and 3 - Control'
csc_data, csc_times, tt_csc = nwb_utils.load_decode_nlx(folder_path=folder_path)

### Use the process_signal object to rereference

In [ ]:
sig = signal_utils.process_signal(tt_csc)
sig.rereference()

### Plot example traces

In [ ]:
fig, axes = plt.subplots(nrows=len(sig.data),ncols=1)



### Using the signal_utils objects

In [ ]:
# TODO: implement signal_utils

# run a filter
sig.butterworth_filter(lowcut = 4, highcut = 12, fs = 32000)

# rereference and plot new results
sig.rereference(rereference_mode = 'CAR')
sig.butterworth_filter(data = sig.signal_rereferenced,lowcut = 4,highcut = 12,fs = 32000)

plt.subplot(3,1,1)
plt.plot(sig.data['TT1a.ncs'][0:32000],'k',linewidth=1)
plt.plot(sig.signal_filtered['TT1a.ncs'][0:32000],'r',linewidth=2)
plt.subplot(3,1,2)
plt.plot(sig.signal_rereferenced['TT1a.ncs'][0:32000],'k',linewidth=1)
plt.plot(sig.signal_filtered['TT1a.ncs'][0:32000],'r',linewidth=2)
plt.subplot(3,1,3)
plt.plot(sig.data['TT1a.ncs'][0:32000],'k',linewidth=1)
plt.plot(sig.signal_rereferenced['TT1a.ncs'][0:32000],'r',linewidth=1)
    

### Between signal correlation

In [ ]:
#%
# testing correlation after signal rereferencing
import pandas as pd
import matplotlib.pyplot as plt
import os

data_pd = pd.DataFrame.from_dict(sig.data)
data_pd_ref = pd.DataFrame.from_dict(sig.signal_rereferenced)

# pairwise correlation
data_cor = pd.DataFrame.corr(self=data_pd, method = 'pearson', min_periods=1, numeric_only = False)
data_cor_reref = pd.DataFrame.corr(self=data_pd_ref, method = 'pearson', min_periods=1, numeric_only = False)

fig, (ax1,ax2) = plt.subplots(nrows=1,ncols=2)
pos1 = ax1.imshow(data_cor)
ax1.set_xticks(range(len(data_cor)), data_cor.columns, rotation='vertical')
ax1.set_yticks(range(len(data_cor)), data_cor.index)
#ax1.yaxis('off')
fig.colorbar(pos1, ax=ax1, shrink=0.5)

pos2 = ax2.imshow(data_cor_reref)
ax2.set_xticks(range(len(data_cor)), data_cor.columns, rotation='vertical')
ax2.set_yticks(range(len(data_cor)), data_cor.index, color='w')
fig.colorbar(pos2, ax=ax2, shrink=0.5)
fig.savefig(folder_path, format='eps')

In [ ]:
data_cor_reref